#Contexte du code

Ce code a été produit par Léo Aguiton et Raphaël Duprez dans le cadre du rendu SIG pour le 01 février 2025.

Ce code permet d'afficher les volcans ainsi que les 500 plus grandes villes du monde.

Pour fonctionner, il a besoin de :
- un Excel des volcans dans le monde. Nous avons utilisé [celui-ci](https://www.kaggle.com/datasets/ramjasmaurya/volcanoes-on-earth-in-2021), trouvable sur Kaggle ;
- un shapefile contenant toutes les villes du monde. Nous avons utilisé [celui-ci](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/), trouvable sur Natural Earth.

Les étapes du code sont :
- Import des bibliothèsques ;
- Import des données ;
- Définition des fonctions pour :
   - Calculer la distance de la ville la plus proche pour un volcan ;
   - Calculer la distance du volcan le plus proche pour une ville ;
- Création des variables nécessaire notamment les villes que nous souhaitons afficher (les 500 plus grandes) ;
- Création de la carte avec des bibliothèques python gratuites et facilement accessible ;
- Export de la carte en .html.

#Import des bibliothèques

In [1]:
# Bibliothèques de manipulation des données
import pandas as pd

# Bibliothèques géospatiales
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from geopy.distance import geodesic

# Bibliothèques de visualisation interactive
import folium
from folium.plugins import MarkerCluster

#Bibliothèque d'export
from google.colab import files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Définition des fonctions

In [3]:
# Fonction pour trouver la ville la plus proche d'un volcan
def nearest_city(volcano, cities):
    volcano_point = Point(volcano['Longitude'], volcano['Latitude'])
    cities_projected = cities.to_crs(epsg=3857)
    volcano_point_projected = gpd.GeoSeries([volcano_point], crs="EPSG:4304").to_crs(epsg=3857).iloc[0]
    cities_projected['distance'] = cities_projected.geometry.distance(volcano_point_projected)
    nearest = cities_projected.loc[cities_projected['distance'].idxmin()]
    return nearest['NAME'], geodesic((volcano['Latitude'], volcano['Longitude']), (nearest['LATITUDE'], nearest['LONGITUDE'])).km

In [4]:
# Fonction pour trouver le volcan le plus proche d'une ville
def nearest_volcano(city, volcanoes):
    city_point = Point(city['LONGITUDE'], city['LATITUDE'])
    volcanoes_projected = volcanoes.to_crs(epsg=3857)
    city_point_projected = gpd.GeoSeries([city_point], crs="EPSG:4304").to_crs(epsg=3857).iloc[0]
    volcanoes_projected['distance'] = volcanoes_projected.geometry.distance(city_point_projected)
    nearest = volcanoes_projected.loc[volcanoes_projected['distance'].idxmin()]
    return nearest['Volcano Name'], geodesic((city['LATITUDE'], city['LONGITUDE']), (nearest['Latitude'], nearest['Longitude'])).km

#Import des données et création des variables

In [5]:
# Chargement des données
volcanoes = pd.read_csv("/content/drive/MyDrive/00_datas/volcanoes around the world in 2021.csv", sep=";")
cities = gpd.read_file("/content/drive/MyDrive/00_datas/villes.shp")

# Sélection des 500 villes les plus peuplées pour l'affichage
cities_display = cities.nlargest(500, "POP2020")

#Création de la carte

In [6]:
# Définition des couleurs par type de volcan pour l'affichage
volcano_colors = {
    'Stratovolcano': '#FF0000',
    'Shield': '#FFA500',
    'Caldera': '#800080',
    'Complex': '#FF1493',
    'Compound': '#FF4500',
    'Cone': '#8B0000'
}
default_color = '#696969'

In [7]:
# Conversion des volcans en GeoDataFrame
geometry = [Point(xy) for xy in zip(volcanoes['Longitude'], volcanoes['Latitude'])]
volcanoes_gdf = gpd.GeoDataFrame(volcanoes, geometry=geometry, crs="EPSG:4304")

In [8]:
# Création de la carte avec plusieurs fonds de carte
m = folium.Map(location=[0, 0], zoom_start=2, tiles=None)

folium.TileLayer('OpenStreetMap', name='OpenStreetMap', attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors').add_to(m)
folium.TileLayer('Stamen Terrain', name='Relief', attr='&copy; <a href="http://maps.stamen.com/">Stamen Design</a>').add_to(m)
folium.TileLayer('CartoDB Positron', name='Fond Clair', attr='&copy; <a href="https://carto.com/attributions">CartoDB</a>').add_to(m)
folium.TileLayer('Esri.WorldImagery', name='Satellite', attr='&copy; <a href="https://www.esri.com/en-us/arcgis/products/arcgis-online/resources">Esri</a>').add_to(m)

In [9]:
#Créé des MarkerCluster pour les villes et les volcans
volcano_marker_cluster = MarkerCluster(name="Volcans").add_to(m)

In [10]:
# Ajout des volcans à la carte avec le calcul de leur distance à la ville la plus proche
for idx, row in volcanoes.iterrows():
    nearest_city_name, nearest_city_distance = nearest_city(row, cities)
    color = volcano_colors.get(row['Type'], default_color)
    popup_content = f"""
                          <div style="width: 300px; white-space: normal;">
                              <p><b>{row['Volcano Name']}</b></p>
                              <p><u>Altitude:</u> {row['Elevation (m)']} m</p>
                              <p><u>Pays:</u> {row['Country']}</p>
                              <p><u>Status:</u> {row['Status']}</p>
                              <p><u>Type:</u> {row['Type']}</p>
                              <p><u>Ville la plus proche:</u> {nearest_city_name} ({nearest_city_distance:.2f} km)</p>
                          </div>
                          """

    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        tooltip=f"{row['Volcano Name']}<br>{row['Elevation (m)']} m<br>{row['Country']}",
        popup=popup_content
    ).add_to(volcano_marker_cluster)

In [12]:
# Ajout des 500 villes les plus peuplées avec la distance du plus proche volcan
for idx, row in cities_display.iterrows():
    nearest_volcano_name, nearest_volcano_distance = nearest_volcano(row, volcanoes_gdf)
    if nearest_volcano_distance < 100:
        city_color = "red"
    elif nearest_volcano_distance < 500:
        city_color = "orange"
    else:
        city_color = "green"
    popup_content_city = f"""
<div style="width: 300px; white-space: normal;">
    <p><b>{row['NAME']}</b></p>
    <p><u>Population (en M habitants):</u> {row['POP2020']}</p>
    <p><u>Volcan le plus proche:</u> {nearest_volcano_name} ({nearest_volcano_distance:.2f} km)</p>
</div>
"""
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=4,
        color=city_color,
        fill=True,
        fill_color=city_color,
        fill_opacity=0.8,
        popup = popup_content_city,
        name="Villes"
    ).add_to(m)

In [13]:
# Ajout des couches à la carte
folium.LayerControl(
    collapsed=True,
    position='topright'
).add_to(m)

In [14]:
# Ajout du titre
title_html = '''<div style="position: fixed; top: 10px; left: 50px; width: 350px; height: 40px; z-index:9999; background-color: white; border-radius: 5px; box-shadow: 0 0 5px rgba(0,0,0,0.2); font-size: 16px; font-weight: bold; padding: 10px; text-align: center;">Distribution Mondiale des Volcans en 2021</div>'''
m.get_root().html.add_child(folium.Element(title_html))

In [15]:
# Création de la légende
legend_html = '''
<div style="position: fixed; bottom: 50px; right: 50px; width: 220px; z-index:9999; background-color: rgba(255, 255, 255, 0.8); border-radius: 5px; box-shadow: 0 0 5px rgba(0,0,0,0.2); font-size: 14px; padding: 10px;">
    <p><strong>Types de Volcans</strong></p>
    <p style="margin: 0;">
        <span style="color: #FF0000; font-size: 16px;">●</span> Stratovolcan<br>
        <span style="color: #FFA500; font-size: 16px;">●</span> Shield<br>
        <span style="color: #800080; font-size: 16px;">●</span> Caldera<br>
        <span style="color: #FF1493; font-size: 16px;">●</span> Complex<br>
        <span style="color: #FF4500; font-size: 16px;">●</span> Compound<br>
        <span style="color: #8B0000; font-size: 16px;">●</span> Cone<br>
        <span style="color: #696969; font-size: 16px;">●</span> Autres types de volcan<br>
        <hr>
        <p><strong>Distance des villes au volcan</strong></p>
        <span style="color: red; font-size: 16px;">●</span> < 100 km<br>
        <span style="color: orange; font-size: 16px;">●</span> 100 - 500 km<br>
        <span style="color: green; font-size: 16px;">●</span> > 500 km
    </p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

In [16]:
# Affichage de la carte
m

#Export de la carte

In [17]:
#Export de la carte
m.save("volcan_dans_le_monde_en_2021.html")
files.download("volcan_dans_le_monde_en_2021.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>